In [1]:
# Read from CSV file.

import pandas as pd

df = pd.read_csv('./clean_text.csv')

print(len(df.index))
df['clean_text'].head()

9079


0    holiday season shopping rush coming social med...
1      police open fire vehicle ram embassy car injury
2    cant thing need financial performance company ...
3                                  bring home kindness
4     wood within striking distance fifth green jacket
Name: clean_text, dtype: object

In [2]:
# Create TF-IDF (1 text).

import numpy as np

tfidf = df['clean_text'][0:1].apply(lambda text: pd.value_counts(text.split(" "))).sum(axis=0).reset_index()
tfidf.columns = ['words', 'tf']
for i, word in enumerate(tfidf['words']):
    tfidf.loc[i, 'idf'] = np.log(df.shape[0] / len(df[df['clean_text'].str.contains(word)]))
tfidf['tf-idf'] = tfidf['tf'] * tfidf['idf']

tfidf.head()

,words,tf,idf,tf-idf
0,shop,1,5.978225,5.978225
1,shopping,1,7.321960,7.321960
2,social,1,5.817882,5.817882
3,rush,1,6.280506,6.280506
4,coming,1,4.208445,4.208445


In [3]:
# Create TF-IDF (all texts).

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(df['clean_text'])
tfidf = pd.DataFrame(matrix.toarray())
tfidf.columns = vectorizer.get_feature_names()

tfidf.head()

,abide,ability,able,aboard,abortion,absence,absolute,absolutely,abstain,abundance,...,yield,york,youd,young,youve,yoy,zero,zip,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
